In [1]:
from helper.trajectory import Trajectory
from models.stonesoup_radar_sim import StonesoupRadarSim
import matplotlib.pyplot as plt

import numpy as np
import pickle
import os
import torch
from models.bnn_ensemble import BNNEnsemble
from models.bnn_ensemble import KalmanFilterUpdater
from helper.evaluator import Evaluator

## Training/Eval Data Import

In [2]:
X_columns = ["measured_x", "measured_y", "measured_z", "measured_vx", "measured_vy", "measured_vz",
             "measured_plus_x", "measured_plus_y", "measured_plus_z", "measured_plus_vx", "measured_plus_vy", "measured_plus_vz", "sigma_range", "sigma_range_rate", "sigma_elevation", "sigma_bearing", "delta_time"]

y_columns = ["truth_x", "truth_y", "truth_z", "truth_vx", "truth_vy", "truth_vz"]

sensor_noise_levels = ['low', 'mid', 'high']
single_noise_level = sensor_noise_levels[2]
num_model_epochs = 1000
num_hidden_nodes = 16

In [3]:
aug_path_in = f"../../dataset/dataframe-readins/cv/augmented/full_traj_data_proc_noise_{single_noise_level}.pkl"

if os.path.exists(aug_path_in):
    with open(aug_path_in, "rb") as e:
        aug_proc_traj_data = pickle.load(e)

In [4]:
def return_index_cv_params(full_proc_traj_data_in):
    train_X_mats = []
    train_y_vecs = []
    test_X_mats = []
    test_y_vecs = []
    
    physics_metadata_train_dicts = []
    physics_metadata_test_dicts = []
    
    data_len = len(full_proc_traj_data_in)
    split_idx = int(0.8 * data_len)
    first_80_indices = np.arange(0, split_idx)
    last_20_indices = np.arange(split_idx, data_len)

    cv_indices = [(first_80_indices, last_20_indices)]
    for index in cv_indices:
    
        train_pmd_dict = {
            'inputPos': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['inputPos'] for key in index[0]], dim=0),
            'inputVel': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['inputVel'] for key in index[0]], dim=0),
            'inputObs': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['inputObs'] for key in index[0]], dim=0),
            'inputRngErr': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['inputRngErr'] for key in index[0]], dim=0),
            'dtVec': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['dtVec'] for key in index[0]], dim=0)}
        
        train_X_mats.append(torch.cat([full_proc_traj_data_in[key]['X_mat'] for key in index[0]], dim=0))
        train_y_vecs.append(torch.cat([full_proc_traj_data_in[key]['y_vec'] for key in index[0]], dim=0))
        physics_metadata_train_dicts.append(train_pmd_dict)
        
        test_pmd_dict = {
            'inputPos': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['inputPos'] for key in index[1]], dim=0),
            'inputVel': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['inputVel'] for key in index[1]], dim=0),
            'inputObs': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['inputObs'] for key in index[1]], dim=0),
            'inputRngErr': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['inputRngErr'] for key in index[1]], dim=0),
            'dtVec': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['dtVec'] for key in index[1]], dim=0)}
        
        test_X_mats.append(torch.cat([full_proc_traj_data_in[key]['X_mat'] for key in index[1]], dim=0))
        test_y_vecs.append(torch.cat([full_proc_traj_data_in[key]['y_vec'] for key in index[1]], dim=0))
        physics_metadata_test_dicts.append(test_pmd_dict)
        
    return train_X_mats, train_y_vecs, test_X_mats, test_y_vecs, physics_metadata_train_dicts, physics_metadata_test_dicts

In [5]:
def train_bnne_model(a_bnne_model, X_mat_in, y_vec_in, load_weight_path=None, noise_level='low', file_prefix_in="ENSEMBLE"):
    trained_model = a_bnne_model
    if type(load_weight_path) == 'str':
        trained_model.load_model_weights(load_weight_path)
    else:
        trained_model.fit(X_mat_in, y_vec_in)
        print(f"FINAL LOSS: {trained_model.loss_values[-1]}")
        trained_model.save_model_weights(file_prefix=file_prefix_in, directory=f'saved_models/ensemble_bnn/{noise_level}')
    return trained_model

In [6]:
def evaluate_model_outputs(y_pred_state, y_pred_covs, y_truth_state):
    TestEval = Evaluator()
        
    full_pred_pos = y_pred_state[:, 0:3]
    full_truth_pos = y_truth_state[:, 0:3].cpu().numpy()
    
    proc_covariances = []    
    for idx, cov_vec in enumerate(y_pred_covs):
        proc_covariances.append(y_pred_covs[idx, 0:3, 0:3])
        
    avg_err = TestEval.calculate_3d_avg_euclid_error(full_truth_pos[:, 0], full_truth_pos[:, 1], full_truth_pos[:, 2], full_pred_pos[:, 0], full_pred_pos[:, 1], full_pred_pos[:, 2])

    avg_md = TestEval.average_mahalanobis_distance(full_pred_pos, proc_covariances, full_truth_pos)
        
    avg_det = TestEval.compute_average_determinant(proc_covariances)
    
    avg_trace = TestEval.compute_average_covariance_trace(proc_covariances)
    
    return avg_err, avg_md, avg_trace, avg_det

## BNN Training

In [7]:
# initialize models
bnne_model_X = BNNEnsemble(input_size=len(X_columns), output_size=1, hidden_layer_size=num_hidden_nodes,
                 prior_sigma_lay=0.01, epochs=num_model_epochs, bias_in=True)

bnne_model_Y = BNNEnsemble(input_size=len(X_columns), output_size=1, hidden_layer_size=num_hidden_nodes,
                 prior_sigma_lay=0.01, epochs=num_model_epochs, bias_in=True)

bnne_model_Z = BNNEnsemble(input_size=len(X_columns), output_size=1, hidden_layer_size=num_hidden_nodes,
                 prior_sigma_lay=0.01, epochs=num_model_epochs, bias_in=True)

bnne_model_VX = BNNEnsemble(input_size=len(X_columns), output_size=1, hidden_layer_size=num_hidden_nodes,
                 prior_sigma_lay=0.01, epochs=num_model_epochs, bias_in=True)

bnne_model_VY = BNNEnsemble(input_size=len(X_columns), output_size=1, hidden_layer_size=num_hidden_nodes,
                 prior_sigma_lay=0.01, epochs=num_model_epochs, bias_in=True)

bnne_model_VZ = BNNEnsemble(input_size=len(X_columns), output_size=1, hidden_layer_size=num_hidden_nodes,
                 prior_sigma_lay=0.01, epochs=num_model_epochs, bias_in=True)

In [8]:
# pull training values
o_train_X_mats, o_train_y_vecs, o_test_X_mats, o_test_y_vecs, o_physics_metadata_train_dicts, o_physics_metadata_test_dicts= return_index_cv_params(aug_proc_traj_data)


X_mat_train = o_train_X_mats[0]
y_vec_train = o_train_y_vecs[0]
X_mat_test = o_test_X_mats[0]
y_vec_test = o_test_y_vecs[0]

### Training Models

In [9]:
%%time
print("TRAINING X PREDICTOR...")
trained_X_model = train_bnne_model(bnne_model_X, X_mat_train, y_vec_train[:, 0:1], load_weight_path=None, noise_level=single_noise_level, file_prefix_in="ENSEMBLE_X")

print("TRAINING Y PREDICTOR...")
trained_Y_model = train_bnne_model(bnne_model_Y, X_mat_train, y_vec_train[:, 1:2], load_weight_path=None, noise_level=single_noise_level, file_prefix_in="ENSEMBLE_Y")

print("TRAINING Z PREDICTOR...")
trained_Z_model = train_bnne_model(bnne_model_Z, X_mat_train, y_vec_train[:, 2:3], load_weight_path=None, noise_level=single_noise_level, file_prefix_in="ENSEMBLE_Z")

print("TRAINING VX PREDICTOR...")
trained_VX_model = train_bnne_model(bnne_model_VX, X_mat_train, y_vec_train[:, 3:4], load_weight_path=None, noise_level=single_noise_level, file_prefix_in="ENSEMBLE_VX")

print("TRAINING VY PREDICTOR...")
trained_VY_model = train_bnne_model(bnne_model_VY, X_mat_train, y_vec_train[:, 4:5], load_weight_path=None, noise_level=single_noise_level, file_prefix_in="ENSEMBLE_VY")

print("TRAINING VZ PREDICTOR...")
trained_VZ_model = train_bnne_model(bnne_model_VZ, X_mat_train, y_vec_train[:, 5:6], load_weight_path=None, noise_level=single_noise_level, file_prefix_in="ENSEMBLE_VZ")

print("DONE TRAINING!")

TRAINING X PREDICTOR...


 Epoch: 1000/1000 | Loss:  61.71 

FINAL LOSS: 61.71078872680664
Model weights saved to saved_models/ensemble_bnn/high\ENSEMBLE_X_bnn_model_2025-06-29_21-20-28.pth
TRAINING Y PREDICTOR...


 Epoch: 1000/1000 | Loss:  61.34 

FINAL LOSS: 61.33757400512695
Model weights saved to saved_models/ensemble_bnn/high\ENSEMBLE_Y_bnn_model_2025-06-29_21-27-27.pth
TRAINING Z PREDICTOR...


...

FINAL LOSS: 2.188448905944824
Model weights saved to saved_models/ensemble_bnn/high\ENSEMBLE_VX_bnn_model_2025-06-29_21-41-20.pth
TRAINING VY PREDICTOR...


 Epoch: 1000/1000 | Loss:   2.00 

FINAL LOSS: 2.0013294219970703
Model weights saved to saved_models/ensemble_bnn/high\ENSEMBLE_VY_bnn_model_2025-06-29_21-47-58.pth
TRAINING VZ PREDICTOR...


 Epoch: 1000/1000 | Loss:   0.56 

FINAL LOSS: 0.5614057183265686
Model weights saved to saved_models/ensemble_bnn/high\ENSEMBLE_VZ_bnn_model_2025-06-29_21-54-30.pth
DONE TRAINING!
CPU times: total: 3h 22min 41s
Wall time: 40min 50s


### Compiling Predictions

In [10]:
%%time
X_test_pred = trained_X_model.predict(X_mat_test)
Y_test_pred = trained_Y_model.predict(X_mat_test)
Z_test_pred = trained_Z_model.predict(X_mat_test)
VX_test_pred = trained_VX_model.predict(X_mat_test)
VY_test_pred = trained_VY_model.predict(X_mat_test)
VZ_test_pred = trained_VZ_model.predict(X_mat_test)

CPU times: total: 1min 16s
Wall time: 16.3 s


In [11]:
print(y_vec_test[0])
print(X_test_pred[0, 0 , :])
print(Y_test_pred[0, 0 , :])
print(Z_test_pred[0, 0 , :])
print(VX_test_pred[0, 0 , :])

tensor([-1.3714e+01,  3.2926e+01,  1.6257e+01, -2.4206e+00,  2.4229e+00,
        -4.1678e-03])
tensor([-17.0465])
tensor([26.0302])
tensor([9.3151])
tensor([-0.4872])


In [12]:
print(X_test_pred.shape)
print(Y_test_pred.shape)
print(Z_test_pred.shape)
print(VX_test_pred.shape)
print(VY_test_pred.shape)
print(VZ_test_pred.shape)

torch.Size([100, 320599, 1])
torch.Size([100, 320599, 1])
torch.Size([100, 320599, 1])
torch.Size([100, 320599, 1])
torch.Size([100, 320599, 1])
torch.Size([100, 320599, 1])


In [13]:
full_prediction = torch.cat((X_test_pred, Y_test_pred, Z_test_pred, VX_test_pred, VY_test_pred, VZ_test_pred), dim=2)
print(full_prediction.shape)

torch.Size([100, 320599, 6])


### Passing to Kalman Filter

In [14]:
# conversion
state_est_mu = full_prediction.mean(dim=0)
x_centered = full_prediction - state_est_mu
state_est_p = torch.einsum('nvc,nvd->vcd', x_centered, x_centered) / (full_prediction.shape[0] - 1)
z_obs, sig_obs = o_physics_metadata_test_dicts[0]['inputObs'], o_physics_metadata_test_dicts[0]['inputRngErr']

In [15]:
kf = KalmanFilterUpdater()
predicted_state, predicted_cov = kf.apply_update(state_est_mu, state_est_p, z_obs, sig_obs)

In [16]:
print(predicted_state.shape)
print(predicted_cov.shape)

(320599, 6)
(320599, 6, 6)


### Running Evaluation

In [17]:
%%time
avg_err, avg_md, avg_trace, avg_det = evaluate_model_outputs(predicted_state, predicted_cov, y_vec_test)

CPU times: total: 3.69 s
Wall time: 5.58 s


In [18]:
print(avg_err)
print(avg_md)
print(avg_trace)
print(avg_det)

9.72220614745404
7.113643883530058
8.241280088759089
8.802782228470575
